In [1]:
import pandas as pd
import numpy as np
import re
import os

from tqdm import tqdm
tqdm.pandas()

/home/barry/miniconda3/envs/bertenv2/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 1 匯入資料

## 1-1 匯入危機公司新聞

### 1-1-1 匯入TEJ危機公司新聞

In [2]:
tej_distress = pd.read_csv('data/2018_2017_tej_distress_0423.csv')
print(tej_distress.shape)
tej_distress.head(2)

(3444, 5)


,公司碼,年月,則次,序號,新聞內容
0,2474 可成,2017/12/29,01,10,可成旗下子公司可成科技( 宿遷) 及可功科技( 宿遷)28 日分別斥資6.2 億元及6.3 ...
1,1611 中電,2017/12/28,01,10,公開資訊觀測站重大訊息公告 ...


### 1-1-2 匯入UDN危機公司新聞

In [3]:
# 設定路徑，將該資料夾內所有檔案名稱列出來
Folder_Path = 'data/udn_distress/'
file_list = os.listdir(Folder_Path)
# file_list

In [4]:
udn_distress = pd.DataFrame(columns=['company', 'title', 'time', 'content'])

for file in file_list:
    df_temp = pd.read_csv('data/udn_distress/' + file)
    udn_distress = pd.concat([udn_distress, df_temp])
udn_distress = udn_distress.drop('title', axis=1).dropna()
print(udn_distress.shape)
udn_distress.head(2)

(2589, 3)


,company,time,content
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（\r\n）積極搶攻電信業自有品牌小筆電市場宣告開花結果。\r\n主管表示，\r\n近期拿下...
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（\r\n）營運出現大轉機，新任總經理許志宏昨（7）日上任，\r\n將再搶下大陸第二階段家電...


### 1-1-3 （針對UDN）擷取日期，把日期格式轉換為yyyy/mm/dd

In [5]:
from datetime import date
import datetime

#測試一下

s = udn_distress['time'].iloc[4]
print(s)
match = re.search('\d+-\d+-\d+', s)
date = str(match.group(0))
date = date.replace('-', '/')
print(date)
date = datetime.datetime.strptime(date, '%Y/%m/%d').date()
# date = datetime.datetime.strptime(match.group(0), '%Y-%m-%d').date()
print(date)

【2009-05-25/經濟日報/C3版/市場焦點】
2009/05/25
2009-05-25


In [6]:
dates = []
for time in udn_distress['time']:
    s = time
    match = re.search('\d+-\d+-\d+', s)
    date = str(match.group(0))
    date = date.replace('-', '/')
    dates.append(date)
udn_distress['date'] = dates
# udn_distress.drop('time', axis=1, inplace=True)
print(udn_distress.shape)
udn_distress.head(2)

(2589, 4)


,company,time,content,date
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（\r\n）積極搶攻電信業自有品牌小筆電市場宣告開花結果。\r\n主管表示，\r\n近期拿下...,2009/04/13
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（\r\n）營運出現大轉機，新任總經理許志宏昨（7）日上任，\r\n將再搶下大陸第二階段家電...,2009/05/08


In [7]:
# 把一些字串刪掉改空格
udn_distress['content'] = udn_distress['content'].astype(str).str.replace('\r' , ' ').str.replace('\t' , ' ').str.replace("\n", " ").str.replace(",", " ").str.replace('"' , ' ').str.replace("'" , ' ')
udn_distress['company'] = udn_distress['company'].astype(str).str.replace('\r' , ' ').str.replace('\t' , ' ').str.replace("\n", " ").str.replace(",", " ").str.replace('"' , ' ').str.replace("'" , ' ')
print(udn_distress.shape)
udn_distress.head(2)

(2589, 4)


,company,time,content,date
0,2331 精英,【2009-04-13/經濟日報/C3版/市場焦點】,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,【2009-05-08/經濟日報/C3版/市場焦點】,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08


In [8]:
udn_distress = udn_distress.loc[:, ['company', 'content', 'date']]
print(udn_distress.shape)
udn_distress.head(2)

(2589, 3)


,company,content,date
0,2331 精英,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08


In [ ]:
udn_distress.to_csv("data/udn_distress_news.csv", index=False)

In [9]:
udn_distress = pd.read_csv("data/udn_distress_news.csv", encoding="utf-8")
print(udn_distress.shape)
udn_distress.head(2)

(2589, 3)


,company,content,date
0,2331 精英,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...,2009/04/13
1,2331 精英,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...,2009/05/08


### 1-1-4 合併危機公司資料集為 distress_news

In [10]:
distress_news = pd.DataFrame(columns=['company', 'date', 'content'])
company = udn_distress['company']
company = company.append(tej_distress['公司碼'])
date = udn_distress['date']
date = date.append(tej_distress['年月'])
content = udn_distress['content']
content = content.append(tej_distress['新聞內容'])
distress_news['company'] = company
distress_news['date'] = date
distress_news['content'] = content
print(distress_news.shape)
distress_news.head(2)

(6033, 3)


,company,date,content
0,2331 精英,2009/04/13,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...
1,2331 精英,2009/05/08,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...


In [ ]:
distress_news.to_csv("data/tej_udn_distress_news.csv", index=False)

In [11]:
distress_news = pd.read_csv("data/tej_udn_distress_news.csv", encoding="utf-8")
print(distress_news.shape)
distress_news.head(2)

(6033, 3)


,company,date,content
0,2331 精英,2009/04/13,（ ）積極搶攻電信業自有品牌小筆電市場宣告開花結果。 主管表示， 近期拿下遠傳（490...
1,2331 精英,2009/05/08,（ ）營運出現大轉機，新任總經理許志宏昨（7）日上任， 將再搶下大陸第二階段家電下鄉大單...


## 1-2 匯入正常公司新聞

### 1-2-1 匯入TFJ正常公司新聞

In [12]:
tej_normal = pd.read_csv('data/2018_2017_tej_normal_0423.csv')
print(tej_normal.shape)
tej_normal.head(2)

(7269, 5)


,公司碼,年月,則次,序號,新聞內容
0,9946 三發地產,2017/12/26,01,10,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,01,10,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...


### 1-2-2 匯入UDN正常公司新聞（0422：正常公司的UDN資料蒐集不完整，暫不採納）

In [ ]:
# Folder_Path = '/home/michael/Documents/M10609101_Thesis/News/udn/normal/'
# file_list = os.listdir(Folder_Path)
# # file_list

In [ ]:
# udn_normal = pd.DataFrame(columns=['company', 'title', 'time', 'content'])

# for file in file_list:
#     df_temp = pd.read_csv('News/udn/normal/' + file)
#     udn_normal = pd.concat([udn_normal, df_temp])
# udn_normal = udn_normal.drop('title', axis=1).dropna()
# udn_normal.head()

In [ ]:
# len(udn_normal)

### 1-2-3 （針對UDN）擷取日期，把日期格式轉換為yyyy/mm/dd（0422：正常公司的UDN資料蒐集不完整，暫不採納）

In [ ]:
# dates = []
# for time in udn_normal['time']:
#     s = time
#     match = re.search('\d+-\d+-\d+', s)
#     date = str(match.group(0))
#     date = date.replace('-', '/')
#     dates.append(date)
# udn_normal['date'] = dates
# # udn_normal.drop('time', axis=1, inplace=True)
# udn_normal.tail()

### 1-2-4 合併危機公司資料集為 normal_news

In [13]:
normal_news = pd.DataFrame(columns=['company', 'date', 'content'])
normal_news['company'] = tej_normal['公司碼']
normal_news['date'] = tej_normal['年月']
normal_news['content'] = tej_normal['新聞內容']
print(normal_news.shape)
normal_news.head(2)

(7269, 3)


,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...


In [ ]:
# normal_news = pd.DataFrame(columns=['company', 'date', 'content'])
# company = udn_normal['company']
# company = company.append(tej_normal['公司碼'])
# date = udn_normal['date']
# date = date.append(tej_normal['年月'])
# content = udn_normal['content']
# content = content.append(tej_normal['新聞內容'])
# normal_news['company'] = company
# normal_news['date'] = date
# normal_news['content'] = content
# normal_news.head()

In [ ]:
normal_news.to_csv("data/tej_normal_news.csv", index=False)

In [14]:
normal_news = pd.read_csv("data/tej_normal_news.csv", encoding="utf-8")
print(normal_news.shape)
normal_news.head(2)

(7269, 3)


,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...


In [15]:
all_news = pd.concat([normal_news, distress_news])   # 上下合併
print(all_news.shape)
all_news.head(2)

(13302, 3)


,company,date,content
0,9946 三發地產,2017/12/26,三發地產(9946)昨(25)日舉行法說會時表示，總銷50億元的高雄「三發晶沙」、總銷50億...
1,2049 上銀,2017/12/11,上銀(2049)公佈11月合併營收首度突破20億元，達20.41億元，月增3.5%、年增34...


In [16]:
all_news['date'] = pd.to_datetime(all_news['date'])
all_news = all_news.sort_values(by=['date']).reset_index(drop=True) # 用post_time排序 # 在重設index
print(all_news.shape)
all_news.head(2)

(13302, 3)


,company,date,content
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...


In [ ]:
all_news.to_csv("data/tej_udn_all_news.csv", index=False)

## 2-3 每間公司風險分布
使用意藍2015年風險資料集、經過BERT主題模型產生的model 將每篇文章輸出1*5的output

13302篇文章就有13302*5

In [1]:
import pandas as pd
import numpy as np
import re
import os

from tqdm import tqdm
tqdm.pandas()

import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, TFBertForSequenceClassification, AdamW

from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense, Embedding, SpatialDropout1D, Dropout, Activation, Flatten, InputLayer

/home/barry/miniconda3/envs/bertenv2/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
all_news = pd.read_csv("data/tej_udn_all_news.csv", encoding="utf-8")
print(all_news.shape)
all_news.head(2)

(13302, 3)


,company,date,content
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...


In [19]:
all_news.describe()

,company,date,content
count,13302,13302,13302
unique,240,3038,12746
top,2330 台積電,2008-01-17,中華信用評等公司昨日確認台塑集團四家核心公司，包括台塑、南亞塑膠、台化、台塑石化，長期企業信...
freq,613,20,4


In [20]:
all_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13302 entries, 0 to 13301
Data columns (total 3 columns):
company    13302 non-null object
date       13302 non-null object
content    13302 non-null object
dtypes: object(3)
memory usage: 311.9+ KB


In [21]:
# 文章字數 > 510了話 去尾
# 字數小於512-2 因為還有CLS SEP
def remove_510(text):
    if len(text) > 510:
        text = text[:510]    # 只取前510個字
    return text

In [22]:
all_news["content"] = all_news["content"].apply(remove_510)
print(all_news.shape)
all_news.head(2)

(13302, 3)


,company,date,content
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...


In [23]:
# Load pre-trained model tokenizer, to convert our text into tokens that correspond to BERT’s vocabulary.
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

def input_ids_all(text):
#    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    tokens = tokenizer.tokenize(text)        # 每個字切詞成一個list
    input_ids = tokenizer.convert_tokens_to_ids(tokens)  # 每個字轉成id
    input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)    # 句子前後加上 CLS SEP 的 id
    input_ids = np.array(input_ids)          # list 轉 numpy
    if len(input_ids) < 512:
        n = 512 - len(input_ids)
        input_ids = np.pad(input_ids, (0, n), mode ='constant', constant_values=(0))  
        # array右邊append n 個 0  補長度到512
    return input_ids

In [24]:
# df['text2'] = df['text']].apply(lambda x: " ".join(jieba.cut(x)))
# df_news['tokens'] = df_news['text'].apply(lambda x:  tokenizer.tokenize(x) )
all_news['input_ids'] = all_news['content'].progress_apply(input_ids_all)
print(all_news.shape)
all_news.head(2)

100%|██████████| 13302/13302 [00:16<00:00, 788.21it/s]


(13302, 4)


,company,date,content,input_ids
0,2316 楠梓電,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,"[101, 1313, 1170, 7442, 6662, 3352, 4496, 3511..."
1,5469 瀚宇博,2007-01-01,印刷電路板產業「大者恆大」趨勢愈來愈明顯，即使景氣連續兩年上升，大廠對於今年不看淡，下游手機...,"[101, 1313, 1170, 7442, 6662, 3352, 4496, 3511..."


In [25]:
# df_news['input_ids'].to_numpy() 出來不是一個2d numpy 
# 只好用for loop一個一個拿出來合併
# 用np vstack超級慢 不知道為何  改用最外層是list append
input_ids = []      # list
for index, row in all_news.iterrows():  
    np_1d = all_news.loc[index,'input_ids']    # 1d np arrary
    input_ids.append(np_1d)                       # 1d np的 list # list[np_1, np_2, np_3, ....]


input_ids = np.array(input_ids)                       # 轉成2d np
print(input_ids.shape)
input_ids

(13302, 512)


array([[ 101, 1313, 1170, ...,    0,    0,    0],
       [ 101, 1313, 1170, ...,    0,    0,    0],
       [ 101, 1313, 1170, ...,    0,    0,    0],
       ...,
       [ 101, 1377, 2768, ...,    0,    0,    0],
       [ 101, 8020, 8021, ...,    0,    0,    0],
       [ 101, 3189, 3315, ...,    0,    0,    0]])

In [26]:
# Recreate the exact same model, including its weights and the optimizer
# 讀取存的model
input_layer = Input(shape = (512,), dtype='int64')  # 預設是float 要改成input_id的int64
load_model = tf.keras.models.load_model('model/model_bert_eland_softmax_1')(input_layer)
model_1 = Model(inputs=input_layer, outputs=load_model)

# Show the model architecture
model_1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
model (Model)                (None, 5)                 104233733 
Total params: 104,233,733
Trainable params: 104,233,733
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time
predictions = model_1.predict(input_ids)   # 輸出的是n*5的編碼值array
print(predictions.shape)
predictions

In [ ]:
df_predictions = pd.DataFrame(predictions, columns = ["c_0", "c_1", "c_2", "c_3", "c_4"])
print(df_predictions.shape)
df_predictions.head(2)

In [ ]:
all_news["c_0"].sum()

In [ ]:
all_news = pd.concat([all_news, df_predictions], axis=1)   # axis=1左右合併 axis=0 上下合併
print(all_news.shape)
all_news.head(2)

### 需要宣告一個240公司*10年的dataframe

In [4]:
company_list = all_news["company"].unique()  # 把欄位unique的值都挑出來變成numpy
print(len(company_list))
company_list

240


array(['2316 楠梓電', '5469 瀚宇博', '3037 欣興', '1612 宏泰', '2498 宏達電',
       '1477 聚陽', '6168 宏齊', '2484 希華', '2534 宏盛', '6277 宏正', '2606 裕民',
       '2527 宏璟', '5305 敦南', '2376 技嘉', '3018 同開', '1236 宏亞', '2536 宏普',
       '3052 夆典', '1558 伸興', '1452 宏益', '2358 廷鑫', '3605 宏致', '2408 南亞科',
       '6271 同欣電', '3015 全漢', '4532 瑞智', '6152 百一', '2331 精英', '1532 勤美',
       '2325 矽品', '3682 亞太電', '8271 宇瞻', '2344 華邦電', '9934 成霖', '8150 南茂',
       '6196 帆宣', '1583 程泰', '3528 安馳', '2511 太子', '8021 尖點', '9911 櫻花',
       '3591 艾笛森', '4919 新唐', '4119 旭富', '1442 名軒', '1618 合機', '1414 東和',
       '1805 寶徠', '1451 年興', '2528 皇普', '2429 銘旺科', '1525 江申', '4414 如興',
       '9943 好樂迪', '2301 光寶科', '2472 立隆電', '3406 玉晶光', '1907 永豐餘',
       '3026 禾伸堂', '6285 啟碁', '2485 兆赫', '2461 光群雷', '2419 仲琦',
       '6281 全國電', '2439 美律', '2340 光磊', '3321 同泰', '2509 全坤建', '1323 永裕',
       '3058 立德', '3004 豐達科', '2330 台積電', '1904 正隆', '5608 四維航',
       '4906 正文', '2392 正崴', '2105 正新', '9921 巨大', '2506 太設', '1507 永大',
 

In [15]:
temp = []
for company_name in company_list:
    for year in range(2008,2018):
        company_name = str(company_name)
        year = int(year)
        temp_np = np.array([company_name , year])    # np(公司名,年分)
        temp.append(temp_np)
        
temp = np.array(temp)
print(len(temp))
temp

2400


array([['2316 楠梓電', '2008'],
       ['2316 楠梓電', '2009'],
       ['2316 楠梓電', '2010'],
       ...,
       ['1453 大將', '2015'],
       ['1453 大將', '2016'],
       ['1453 大將', '2017']], dtype='<U11')

In [16]:
df_topic_5_prob = pd.DataFrame(temp, columns = ['company' , 'date'])
print(df_topic_5_prob.shape)
df_topic_5_prob

(2400, 2)


,company,date
0,2316 楠梓電,2008
1,2316 楠梓電,2009
2,2316 楠梓電,2010
3,2316 楠梓電,2011
4,2316 楠梓電,2012
...,...,...
2395,1453 大將,2013
2396,1453 大將,2014
2397,1453 大將,2015
2398,1453 大將,2016


In [17]:
df_topic_5_prob["date"] = df_topic_5_prob["date"].astype(int)
df_topic_5_prob["company"] = df_topic_5_prob["company"].astype(str)
df_topic_5_prob["c_0"] = 0.
df_topic_5_prob["c_1"] = 0.
df_topic_5_prob["c_2"] = 0.
df_topic_5_prob["c_3"] = 0.
df_topic_5_prob["C-4"] = 0.
print(df_topic_5_prob.shape)
df_topic_5_prob.head(2)

(2400, 7)


,company,date,c_0,c_1,c_2,c_3,C-4
0,2316 楠梓電,2008,0.0,0.0,0.0,0.0,0.0
1,2316 楠梓電,2009,0.0,0.0,0.0,0.0,0.0


In [18]:
# a column contains "A" & b column contains "B" 
df_topic_5_prob[(df_topic_5_prob["company"].str.contains("楠梓電") & df_topic_5_prob["date"].astype(str).str.contains("2009") )]

,company,date,c_0,c_1,c_2,c_3,C-4
1,2316 楠梓電,2009,0.0,0.0,0.0,0.0,0.0
